# 🔥 Fine-Tune LLMs with Fireworks AI

This notebook guides you through fine-tuning a Large Language Model using Fireworks AI.

**What you'll learn:**
1. How to prepare a dataset for fine-tuning
2. How to upload data to Fireworks
3. How to start and monitor a fine-tuning job
4. How to test your fine-tuned model

**Prerequisites:**
- A Fireworks AI account (free tier available at https://fireworks.ai)
- An API key from Fireworks

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install fireworks-ai openai datasets pandas

In [ ]:
import os
import json
import pandas as pd
from google.colab import userdata

# Set your Fireworks API key
# Option 1: Use Colab secrets (recommended)
try:
    FIREWORKS_API_KEY = userdata.get('FIREWORKS_API_KEY')
except:
    # Option 2: Enter manually
    FIREWORKS_API_KEY = input("Enter your Fireworks API key: ")

os.environ['FIREWORKS_API_KEY'] = FIREWORKS_API_KEY
print("✅ API key configured")

## 2. Dataset Preparation

Fireworks expects data in **JSONL format** with conversation-style messages.

### Format Options:

**Chat format (recommended):**
```json
{"messages": [{"role": "system", "content": "..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```

**Completion format:**
```json
{"prompt": "...", "completion": "..."}
```

In [ ]:
# Example: Create a sample dataset for a customer support assistant
# Replace this with your own data!

sample_data = [
    {
        "messages": [
            {"role": "system", "content": "You are a helpful customer support assistant for a software company."},
            {"role": "user", "content": "How do I reset my password?"},
            {"role": "assistant", "content": "To reset your password:\n1. Go to the login page\n2. Click 'Forgot Password'\n3. Enter your email address\n4. Check your inbox for a reset link\n5. Follow the link to create a new password\n\nThe link expires in 24 hours. Need any other help?"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a helpful customer support assistant for a software company."},
            {"role": "user", "content": "My subscription expired, how do I renew?"},
            {"role": "assistant", "content": "I'd be happy to help you renew your subscription!\n\n1. Log into your account dashboard\n2. Navigate to 'Billing & Subscription'\n3. Click 'Renew' or 'Upgrade'\n4. Select your preferred plan\n5. Complete the payment\n\nYour access will be restored immediately after payment. Would you like me to walk you through any specific step?"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a helpful customer support assistant for a software company."},
            {"role": "user", "content": "I'm getting an error when trying to upload files"},
            {"role": "assistant", "content": "I'm sorry to hear you're experiencing upload issues. Let me help troubleshoot:\n\n**Common causes:**\n- File exceeds size limit (max 100MB)\n- Unsupported file format\n- Browser cache issues\n\n**Try these steps:**\n1. Check if your file is under 100MB\n2. Ensure it's a supported format (.pdf, .doc, .xlsx, .png, .jpg)\n3. Clear browser cache and cookies\n4. Try a different browser\n\nIf the issue persists, could you share the exact error message? That will help me identify the specific problem."}
        ]
    }
]

print(f"📊 Sample dataset with {len(sample_data)} examples")
print("\nExample entry:")
print(json.dumps(sample_data[0], indent=2))

### 2.1 Create Your Own Dataset

Here's where you define your training examples. The quality and diversity of your data directly impacts model performance.

**Tips for good training data:**
- Include 50-1000+ examples for best results
- Cover diverse scenarios your model will encounter
- Maintain consistent formatting in responses
- Include edge cases and challenging examples

In [ ]:
# ============================================================
# TODO: Replace this with YOUR training data
# ============================================================

def create_training_example(system_prompt: str, user_input: str, assistant_response: str) -> dict:
    """
    Helper function to create a single training example.
    
    Args:
        system_prompt: The system instruction that sets the assistant's behavior
        user_input: What the user says/asks
        assistant_response: The ideal response you want the model to learn
    
    Returns:
        A properly formatted training example dict
    """
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": assistant_response}
        ]
    }

# Your system prompt - describes the assistant's persona/behavior
SYSTEM_PROMPT = "You are a helpful customer support assistant for a software company."

# Add your training examples here
training_data = [
    create_training_example(
        system_prompt=SYSTEM_PROMPT,
        user_input="How do I cancel my account?",
        assistant_response="I understand you'd like to cancel your account. Before proceeding, may I ask what's prompting this decision? I'd like to see if there's anything we can do to address your concerns.\n\nIf you'd still like to proceed:\n1. Go to Account Settings\n2. Select 'Subscription'\n3. Click 'Cancel Subscription'\n4. Complete the confirmation\n\nNote: You'll retain access until the end of your billing period."
    ),
    # Add more examples...
]

# Combine with sample data for this demo
all_training_data = sample_data + training_data
print(f"📊 Total training examples: {len(all_training_data)}")

In [ ]:
# Alternative: Load from CSV or existing data source

def load_from_csv(csv_path: str, system_prompt: str) -> list:
    """
    Load training data from a CSV file.
    
    Expected CSV columns: 'user_input', 'assistant_response'
    """
    df = pd.read_csv(csv_path)
    data = []
    for _, row in df.iterrows():
        data.append({
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": row['user_input']},
                {"role": "assistant", "content": row['assistant_response']}
            ]
        })
    return data

# Uncomment to use:
# training_data = load_from_csv('your_data.csv', SYSTEM_PROMPT)

In [ ]:
# Save training data to JSONL file

TRAIN_FILE = 'training_data.jsonl'

with open(TRAIN_FILE, 'w') as f:
    for example in all_training_data:
        f.write(json.dumps(example) + '\n')

print(f"✅ Saved {len(all_training_data)} examples to {TRAIN_FILE}")

# Verify the file
!head -n 1 {TRAIN_FILE}

## 3. Upload Dataset to Fireworks

Now we'll upload our training data to Fireworks AI.

In [ ]:
import requests

def upload_dataset(file_path: str, dataset_name: str) -> str:
    """
    Upload a JSONL file to Fireworks as a dataset.
    
    Returns the dataset ID for use in fine-tuning.
    """
    url = "https://api.fireworks.ai/v1/datasets"
    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}"
    }
    
    with open(file_path, 'rb') as f:
        files = {
            'file': (file_path, f, 'application/jsonl')
        }
        data = {
            'name': dataset_name
        }
        response = requests.post(url, headers=headers, files=files, data=data)
    
    if response.status_code == 200:
        result = response.json()
        print(f"✅ Dataset uploaded successfully!")
        print(f"   Dataset ID: {result.get('id')}")
        return result.get('id')
    else:
        print(f"❌ Upload failed: {response.status_code}")
        print(response.text)
        return None

# Upload the dataset
DATASET_NAME = "customer-support-v1"  # Change this to your preferred name
dataset_id = upload_dataset(TRAIN_FILE, DATASET_NAME)

## 4. Start Fine-Tuning Job

Now let's start the fine-tuning job. Fireworks supports several base models:

| Model | Description | Best For |
|-------|-------------|----------|
| `llama-v3p1-8b-instruct` | Meta Llama 3.1 8B | General purpose, fast |
| `llama-v3p1-70b-instruct` | Meta Llama 3.1 70B | Higher quality, slower |
| `mistral-7b-instruct-v0p2` | Mistral 7B | Good balance |
| `mixtral-8x7b-instruct` | Mixtral 8x7B MoE | Complex tasks |

In [ ]:
def create_fine_tuning_job(
    dataset_id: str,
    model_name: str,
    base_model: str = "accounts/fireworks/models/llama-v3p1-8b-instruct",
    epochs: int = 3,
    learning_rate: float = 1e-5
) -> dict:
    """
    Create a fine-tuning job on Fireworks.
    
    Args:
        dataset_id: ID of the uploaded dataset
        model_name: Name for your fine-tuned model
        base_model: The base model to fine-tune
        epochs: Number of training epochs
        learning_rate: Learning rate for training
    """
    url = "https://api.fireworks.ai/v1/fine-tuning/jobs"
    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": base_model,
        "dataset": dataset_id,
        "output_model_name": model_name,
        "hyperparameters": {
            "n_epochs": epochs,
            "learning_rate": learning_rate
        }
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        result = response.json()
        print(f"✅ Fine-tuning job created!")
        print(f"   Job ID: {result.get('id')}")
        print(f"   Status: {result.get('status')}")
        return result
    else:
        print(f"❌ Job creation failed: {response.status_code}")
        print(response.text)
        return None

In [ ]:
# ============================================================
# Configure your fine-tuning job
# ============================================================

# Your fine-tuned model's name
MODEL_NAME = "customer-support-llama"  # Change this!

# Base model selection
BASE_MODEL = "accounts/fireworks/models/llama-v3p1-8b-instruct"

# Training hyperparameters
EPOCHS = 3  # More epochs = more training, but risk of overfitting
LEARNING_RATE = 1e-5  # Lower = more stable, higher = faster learning

# Start the job (only run when ready!)
if dataset_id:
    job = create_fine_tuning_job(
        dataset_id=dataset_id,
        model_name=MODEL_NAME,
        base_model=BASE_MODEL,
        epochs=EPOCHS,
        learning_rate=LEARNING_RATE
    )
else:
    print("⚠️ No dataset ID - please upload your dataset first")

## 5. Monitor Training Progress

In [ ]:
import time

def check_job_status(job_id: str) -> dict:
    """Check the status of a fine-tuning job."""
    url = f"https://api.fireworks.ai/v1/fine-tuning/jobs/{job_id}"
    headers = {
        "Authorization": f"Bearer {FIREWORKS_API_KEY}"
    }
    
    response = requests.get(url, headers=headers)
    return response.json()

def monitor_job(job_id: str, poll_interval: int = 60):
    """Monitor a fine-tuning job until completion."""
    print(f"📊 Monitoring job {job_id}...")
    print("   (This may take several minutes to hours depending on dataset size)\n")
    
    while True:
        status = check_job_status(job_id)
        state = status.get('status', 'unknown')
        
        print(f"   Status: {state}")
        
        if state in ['succeeded', 'completed']:
            print("\n✅ Fine-tuning completed successfully!")
            print(f"   Model: {status.get('fine_tuned_model')}")
            return status
        elif state in ['failed', 'cancelled']:
            print(f"\n❌ Job {state}")
            print(f"   Error: {status.get('error', 'Unknown error')}")
            return status
        
        time.sleep(poll_interval)

# Monitor the job (uncomment when you have a job running)
# job_status = monitor_job(job['id'])

In [ ]:
# Quick status check (use this to check status without waiting)
if 'job' in dir() and job:
    status = check_job_status(job['id'])
    print(f"Job ID: {job['id']}")
    print(f"Status: {status.get('status')}")
    print(f"Model: {status.get('fine_tuned_model', 'Not ready yet')}")

## 6. Test Your Fine-Tuned Model

Once training completes, test your model!

In [ ]:
from openai import OpenAI

# Initialize Fireworks client (uses OpenAI-compatible API)
client = OpenAI(
    api_key=FIREWORKS_API_KEY,
    base_url="https://api.fireworks.ai/inference/v1"
)

def chat_with_model(model_id: str, user_message: str, system_prompt: str = None) -> str:
    """
    Send a message to your fine-tuned model.
    
    Args:
        model_id: Your fine-tuned model ID
        user_message: The user's input
        system_prompt: Optional system prompt
    """
    messages = []
    
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    messages.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        max_tokens=512,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
# ============================================================
# Test your fine-tuned model
# ============================================================

# Replace with your actual fine-tuned model ID
FINE_TUNED_MODEL = f"accounts/your-account/models/{MODEL_NAME}"

# Test queries
test_queries = [
    "How do I export my data?",
    "Why is the app running slowly?",
    "Can I get a refund?"
]

print("🧪 Testing fine-tuned model\n")
print("=" * 60)

for query in test_queries:
    print(f"\n👤 User: {query}")
    print("-" * 40)
    
    try:
        response = chat_with_model(
            model_id=FINE_TUNED_MODEL,
            user_message=query,
            system_prompt=SYSTEM_PROMPT
        )
        print(f"🤖 Assistant: {response}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("=" * 60)

## 7. Compare Base vs Fine-Tuned Model

See the difference your fine-tuning made!

In [ ]:
def compare_models(query: str, system_prompt: str = None):
    """Compare responses from base and fine-tuned models."""
    print(f"Query: {query}\n")
    print("=" * 60)
    
    # Base model response
    print("\n📌 BASE MODEL (Llama 3.1 8B):")
    print("-" * 40)
    try:
        base_response = chat_with_model(
            model_id="accounts/fireworks/models/llama-v3p1-8b-instruct",
            user_message=query,
            system_prompt=system_prompt
        )
        print(base_response)
    except Exception as e:
        print(f"Error: {e}")
    
    # Fine-tuned model response
    print(f"\n🎯 FINE-TUNED MODEL ({MODEL_NAME}):")
    print("-" * 40)
    try:
        ft_response = chat_with_model(
            model_id=FINE_TUNED_MODEL,
            user_message=query,
            system_prompt=system_prompt
        )
        print(ft_response)
    except Exception as e:
        print(f"Error: {e}")

# Compare on a test query
compare_models(
    query="I can't log in to my account",
    system_prompt=SYSTEM_PROMPT
)

## 📚 Additional Resources

- [Fireworks AI Documentation](https://docs.fireworks.ai/)
- [Fine-Tuning Guide](https://docs.fireworks.ai/fine-tuning/fine-tuning-guide)
- [Model Catalog](https://fireworks.ai/models)
- [Pricing](https://fireworks.ai/pricing)

## 💡 Tips for Better Results

1. **More data is better**: Aim for 100+ high-quality examples
2. **Consistency matters**: Keep response style consistent across examples
3. **Cover edge cases**: Include examples of tricky scenarios
4. **Iterate**: Start small, test, then expand your dataset
5. **Monitor metrics**: Track loss during training for signs of overfitting